# Librerias necesarias

In [38]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta
import sqlalchemy as sa
from configparser import ConfigParser

# Funciones

 En el presente script se almacenan las funciones que serán utilizadas a lo largo del proyecto, deberán ser importadas en los scripts para que cumplan su funcionalidad.

 Detalle de las funciones:

*   **get_data(base_url,endpoint,params)**: Realiza una solicitud GET para obtener datos. Hace uso de parámetros para lograr la conexion.
*   **build_table(json_data)**: Construye objetos DataFrame a partir de datos en formato JSON.
*   **date_fromTo()**: Funcion que retorna un intervalo de tiempo. Si no recibe parámetros: retorna parámetros desde el primero de agosto del año 2023 hasta la fecha actual bajo el estándar de tiempo UTC. Si recibe parámetros, deben ser el número de dias anteriores a la consulta.
*   **save_to_csv()**: Funcion que guarda un DataFrame en formato .csv, en una ubicacion establecida.
*   **agg_date()**: Funcion que toma 2 parametros (diccionario/fecha). Retorna una fecha
*   **agg_code()**: Funcion que toma 2 parametros (diccionario/codigo). Retorna un codigo.
*   **connect_to_postgres()**: Funcion que realiza la conexion a la base de datos OLAP.

In [39]:
def get_data(base_url, endpoint, params= None):
  try:
    if params == None:
      endpoint_url = f"{base_url}/{endpoint}"
    else:
      endpoint_url = f"{base_url}/{endpoint}/{params['start']}/{params['end']}" # Condicional creada en el caso que un endpoint requiera parámetros de tiempo.

    response = requests.get(endpoint_url, params=params)
    response.raise_for_status()  # Levanta una excepción si hay un error en la respuesta HTTP.

    try: # Verificar si los datos están en formato JSON.
          data = response.json()
          data = data["data"]
    except:
        print("El formato de respuesta no es el esperado")
        return None
    return data
  
  except requests.exceptions.RequestException as e: # Capturar cualquier error de solicitud, como errores HTTP.
      print(f"La petición ha fallado. Código de error : {e}")
      return None

In [40]:
def build_table(json_data):
  df = pd.json_normalize(json_data)
  return df

In [41]:
def date_fromTo(number_days = None):
  if number_days == None:
    end_date = datetime.utcnow() - timedelta(days=1)
    start_date = datetime.utcnow() - timedelta(days=31) # Fecha de inicio: 31 dias antes que la ejecucion del script.
  else:
    end_date = datetime.utcnow() - timedelta(days=1)
    start_date = datetime.utcnow() - timedelta(number_days)

  end_date = end_date.strftime("%Y-%m-%dT%H:00:00Z")
  start_date = start_date.strftime("%Y-%m-%dT%H:00:00Z")

  params = {
    "start": start_date,
    "end": end_date
    }
  return params

In [42]:
def save_to_csv(df, save_path):
  
  """
  Funcion que almacena un DataFrame en formato .csv, en una ubicacion especificada.
  
  Argumentos:
  
    df= DataFrame pandas que se desea guardar.
    save_path= ruta donde se desea guardar el archivo.csv, en formato de texto.
    
  Excepciones:
  
    TypeError= Si el argumento df no es un DataFrame de pandas.
  """
  
  # Verificar que df sea un DataFrame.
  if not isinstance(df, pd.DataFrame):
    raise TypeError('el argumento "df" debe ser un DataFrame de pandas. ')

  # Crear el directorio si no existe
  directory = os.path.dirname(save_path)
  if directory and not os.path.exists(directory):
    os.makedirs(directory)

  try:
    #Guardar el DataFrame en formato csv
    df.to_csv(save_path, index=False)
    print('DataFrame guardado exitosamente en formato CSV.')
  except Exception as e:
    print(f'Error al guardar el DataFrame en formato CSV: {str(e)}')


In [43]:
def agg_date(dict, date):
  dict['date'] = date
  return date

In [44]:
def agg_code(dict, code):
  dict['cod_date'] = code
  return code

In [45]:
#Funcion que establece conexion a base de datos postgresql

def connect_to_postgres(config_file_path= 'config.ini', section= 'postgres'):

  """
  Argumentos:
    config_file_path (str): Ruta del archivo de configuracion INI. Por defecto, "config.ini".
    section (str, optional): Nombre de la seccion en el archivo INI que contiene los datos de conexion. Por defecto, "postgres".

  Returns:
    sqlalchemy.engine.Engine: La conexion a la base de datos.
  """
  # Comprobar si el archivo de configuracion existe.
  if not os.path.exists(config_file_path):
    raise FileNotFoundError(f'El archivo de configuracion "{config_file_path}" no existe.')

  # Leer la configuracion desde el archivo INI.
  config = ConfigParser()
  config.read(config_file_path)
  conn_data = config[section]

  # Obtener los parametros de conexion:
  host = conn_data.get('host')
  port = conn_data.get('port')
  db = conn_data.get('db')
  user = conn_data.get('user')
  pwd = conn_data.get('pwd')

  url = f"postgresql://{user}:{pwd}@{host}:{port}/{db}"

  # Establecer la conexion a la base de datos PostgreSQL
  conn = sa.create_engine(url)
  return conn
